# Lecture 3: Password Attack with Difference Analysis - Analyse

In this example we want to improve the basic passwdcheck to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the basic passwdcheck:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_correct = 0;
        break;
    }
}
```

The timing attack discussed in the last example worked because the loop's runtime varies with the number of correct characters. Once the first wrong character occurs the loop breaks.
This is, what we want to change:

```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_correct = 0;
    }
}
```

This is an excerpt from `3_password_fixed.c`. It is clear that the loop does not break after the first wrong character and always all characters of the password are checked.

## MAD password attack

Given two traces $t_1, t_2$ their MAD value is defined as:
$$\text{MAD}(t_1, t_2) := \text{max}\big( \big| t_1[i] - t_2[i] \big| \big),$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./3_password_fixed.c')

In [ ]:
import struct
import time
import warnings
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write('p', attempt.encode('iso-8859-1') + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read('p', 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10
from bokeh.models import Span, Label, BoxAnnotation

output_notebook()

In [ ]:
def mad(trace1, trace2):
    return max(abs(trace1 - trace2))

## Analysis of differences

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
Have a look at the assembly code of `3_password_fixed.c`. It's the `.lss` file. Understand instruction per instruction what the CPU is doing. The [XMEGA reference manual](http://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-8210-8-and-16-bit-AVR-Microcontrollers-XMEGA-D_Manual.pdf) might be helpful.
</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>
    
Create a plot to explain your investigations and especially their relation to the MAD-Attack. The plot below is an example. The manual of [bokeh](https://docs.bokeh.org/en/latest/index.html) might be helpful. I used `line`, `Span`, `Label`.
    
<img src="Lecture 3: Password Attack with Difference Analysis - Analyse.png"></img>
</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 3</div>
   
Explain why
- At "ins 3" the green line is much higher than the others
- At "ins 3" the others are not equal in height
- The differences at "ins 1" are quite high

</div>